# Example 3: Multi-Frame Overlap

This example shows how to work with more than 2 overlapping frames and explore different frame configurations.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from frame_overlap import Data, Reconstruct, Workflow

%matplotlib inline

## 3-Frame Overlap Example

In [ ]:
# Create 3-frame overlap: frames start at 0, 15, and 30 ms
wf_3frame = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)

(wf_3frame
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 15, 30], total_time=45)
    .reconstruct(kind='wiener', noise_power=0.01))

print(f"3-frame overlap created")
print(f"Kernel: {wf_3frame.data.kernel}")
print(f"Total time: 45 ms")
print(f"Reconstruction χ²/dof: {wf_3frame.recon.statistics['chi2_per_dof']:.2f}")

## Visualize 3-Frame Overlap

In [ ]:
# Plot overlapped transmission
wf_3frame.data.plot(kind='transmission', show_errors=False, ylim=(0, 1), xlim=(0, 45))
plt.title('3-Frame Overlap: Transmission')

# Add vertical lines showing frame boundaries
for frame_start in [0, 15, 30]:
    plt.axvline(frame_start, color='red', linestyle='--', alpha=0.3)
plt.show()

# Plot signal stages
wf_3frame.data.plot(kind='signal', show_stages=True, show_errors=False, ylim=(0, 8e3))
plt.title('3-Frame Overlap: Signal Processing Stages')
plt.show()

## Reconstruction Quality

In [ ]:
# Plot reconstruction
wf_3frame.recon.plot(kind='transmission', show_errors=False, ylim=(0, 1))
plt.show()

# Print statistics
print("Reconstruction Statistics:")
for key, value in wf_3frame.recon.get_statistics().items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")

## 4-Frame Overlap Example

In [ ]:
# Create 4-frame overlap: frames at 0, 12, 24, 36 ms
wf_4frame = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)

(wf_4frame
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 12, 24, 36], total_time=48)
    .reconstruct(kind='wiener', noise_power=0.01))

print(f"\n4-frame overlap created")
print(f"Kernel: {wf_4frame.data.kernel}")
print(f"Total time: 48 ms")
print(f"Reconstruction χ²/dof: {wf_4frame.recon.statistics['chi2_per_dof']:.2f}")

In [ ]:
# Compare overlapped signals
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 3-frame
time_3 = wf_3frame.data.overlapped_data['time'] / 1000  # Convert to ms
signal_3 = wf_3frame.data.overlapped_data['counts']
axes[0].plot(time_3, signal_3, drawstyle='steps-mid')
axes[0].set_xlim(0, 45)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Counts')
axes[0].set_title('3-Frame Overlap')
axes[0].grid(True, alpha=0.3)
for frame in [0, 15, 30]:
    axes[0].axvline(frame, color='red', linestyle='--', alpha=0.3)

# 4-frame
time_4 = wf_4frame.data.overlapped_data['time'] / 1000
signal_4 = wf_4frame.data.overlapped_data['counts']
axes[1].plot(time_4, signal_4, drawstyle='steps-mid', color='orange')
axes[1].set_xlim(0, 48)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Counts')
axes[1].set_title('4-Frame Overlap')
axes[1].grid(True, alpha=0.3)
for frame in [0, 12, 24, 36]:
    axes[1].axvline(frame, color='red', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## Compare Reconstruction Quality

More overlapping frames generally make reconstruction harder (higher chi-squared).

In [ ]:
# Create 2-frame for comparison
wf_2frame = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)
(wf_2frame
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .reconstruct(kind='wiener', noise_power=0.01))

# Compare statistics
configs = [
    ('2-frame', wf_2frame),
    ('3-frame', wf_3frame),
    ('4-frame', wf_4frame)
]

print("\nReconstruction Quality Comparison:")
print("-" * 60)
print(f"{'Config':<12} {'Frames':<8} {'χ²/dof':<12} {'RMSE':<12}")
print("-" * 60)
for name, wf in configs:
    stats = wf.recon.get_statistics()
    n_frames = len(wf.data.kernel)
    chi2 = stats['chi2_per_dof']
    rmse = stats['rmse']
    print(f"{name:<12} {n_frames:<8} {chi2:<12.2f} {rmse:<12.2e}")

## Optimize for Multi-Frame Setup

Find optimal noise_power for 3-frame overlap.

In [ ]:
# Sweep noise_power for 3-frame overlap
results_3frame = (Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 15, 30], total_time=45)
    .groupby('noise_power', low=0.05, high=0.3, num=10)
    .reconstruct(kind='wiener')
    .analyze(xs='iron', vary_background=True, vary_response=True)
    .run())

results_3frame_clean = results_3frame.dropna(subset=['redchi2'])
best_3frame = results_3frame_clean.loc[results_3frame_clean['redchi2'].idxmin()]

print(f"\nOptimal for 3-frame overlap:")
print(f"  noise_power: {best_3frame['noise_power']:.4f}")
print(f"  χ²/dof: {best_3frame['redchi2']:.2f}")

# Plot
results_3frame_clean.plot(x='noise_power', y='redchi2', marker='o', style='-o')
plt.axvline(best_3frame['noise_power'], color='red', linestyle=':', alpha=0.7)
plt.xlabel('Noise Power')
plt.ylabel('Reduced χ²')
plt.title('3-Frame Overlap: Optimal Noise Power')
plt.grid(True, alpha=0.3)
plt.show()

## Key Takeaways

- **Multi-frame support**: Can handle 2, 3, 4, or more overlapping frames
- **Frame spacing**: Kernel defines start times for each frame
- **Reconstruction difficulty**: More frames → harder reconstruction (higher χ²)
- **Parameter optimization**: Optimal noise_power may differ for different frame counts
- **Flexible configuration**: Use `total_time` to control overall time range